# Notebook to Convert AmsterdamUMCdb to MIMIC-III Format

Here we will be converting the [AmsterdamUMCdb](https://github.com/AmsterdamUMC/AmsterdamUMCdb) data files to the MIMIC-III data file format as generated by [MIMIC-Code](https://github.com/MIT-LCP/mimic-code). We do this as to allow the exact same preprocessing pipeline to be applied to both MIMIC and the AmsterdamUMCdb.

In [25]:
# Dependencies
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm

# Directory where AmsterdamUMCdb source files are kept
DATA_DIR = '../data/amsterdam-umc-db/'

if not os.path.exists(DATA_DIR):
    raise Exception('Dataset directory %s does not exist!' % DATA_DIR)

# Directory where AmsterdamUMCdb is kept
OUTPUT_DIR = '../data/amsterdam-umc-db/final/'

if not os.path.exists(DATA_DIR):
    os.makedirs(OUTPUT_DIR)
    
# Batch size for large files
BATCH_SIZE = 500

---
## Admissions


In [ ]:
admission_df = pd.read_csv(DATA_DIR + 'admissions.csv', usecols=['admissionid', 'patientid', 'destination'])
print('Num admissions:', len(admission_df.index))

# Check for DUPLICATES!
print("Number of duplicated Metavision admissions: %d" % sum(admission_df.duplicated('patientid', keep=False) == True))

# List of all admission IDs
admission_list = admission_df['admissionid']
admission_list

# Assign admissions to batches for large files
admission_batches = [admission_list[i: i + BATCH_SIZE] for i in range(0, len(admission_list), BATCH_SIZE)] 

---
## Cohort: `suspected_infection_time_poe`, `window_start`, `window_end` times

In [21]:
antibiotics_ordercategoryids = [
                                15, # 'Injecties Antimicrobiele middelen'
                                21  # 'Niet iv Antimicrobiele middelen' 
                               ]

In [ ]:
# Rename admissions_df
admission_df.rename({'admissionid': 'icustay_id',
                     'patientid': 'subject_id',
                     'destination': 'hospital_expire_flag'})

# In-hospital mortality (1 if deceased; if survived)
admission_df['hospital_expire_flag'] = (admission_df['hospital_expire_flag'] == 'Overleden')

# Suspected Infection Time (start/continuation of antibiotics excl. prophylactic regimens)
antibiotics_df = pd.read_csv('drugitems.csv', usecols=['admissionid', 'ordercategoryid', 'start'])
antibiotics_df = antibiotics_df[antibiotics_df['ordercategoryid'].isin(antibiotics_ordercategoryids)] # TODO: identify preventive/prophylactic treatments?
suspected_infection_df = antibiotics_df[['admissionid', 'start']].groupby('admissionid').first()      # First antibiotic administration!
suspected_infection_df.rename({'admissionid': 'icustay_id',
                               'start': 'suspected_infection_time_poe'}, inplace=True)                # Rename columns to MIMIC-III namespace
del antibiotics_df

# Left-join into one DataFrame!
cohort_df = admission_df.join(suspected_infection_df, how='left', on='icustay_id')

# Convert infection time to timestamp since admission (starttime of first admission is today for simplicity)
cohort_df['suspected_infection_time_poe'] = pd.to_timedelta(cohort_df['suspected_infection_time_poe'], unit='ms') + pd.Timestamp('today')

# Define window_start and window_end times
cohort_df['window_start'] = cohort_df['suspected_infection_time_poe'] - pd.Timedelta(days=1)
cohort_df['window_end'] = cohort_df['suspected_infection_time_poe'] + pd.Timedelta(days=2)

print('Done!')

#### Save as `cohort.csv`

In [ ]:
# cohort.csv -> icustay_id, suspected_infection_time_poe (?), window_start, window_end, hospital_expire_flag
cohort_columns = ['icustay_id', 'suspected_infection_time_poe', 'window_start', 'window_end', 'hospital_expire_flag']
cohort_df[cohort_columns].to_csv(OUTPUT_DIR + 'cohort.csv');
print('Done!')

---
## Demographics

In [ ]:
# Age/weight/height group definitions to group centroids
age_map = {'18-39': 30,
           '40-49': 45, 
           '50-59': 55,
           '60-69': 65, 
           '70-79': 75, 
           '80+': 85}  # Arbitrary

weight_map = {'59-': 55,
              '60-69': 65, 
              '70-79': 75,
              '80-89': 85, 
              '90-99': 95, 
              '100-109': 105, 
              '110+': 115,  
              'N/A': 77.3}  # Dutch average weight
  
height_map = {'159-': 155,
              '160-169': 165, 
              '170-179': 175,
              '180-189': 185, 
              '190+': 195, 
              'N/A': 176.1} # Average height of men and women in NL

In [ ]:
# icustay_ids 
icustay_id = patient_df['admissionid']

# Age
age = patient_df['agegroup'].map(age_map)

# Gender
is_male = patient_df['gender'] == 'Man'

# Weight
weight = patient_df['weightgroup'].apply(weight_map)

# Height
height = patient_df['heightgroup'].apply(height_map)

print('Done!')

#### Ventilator

In [ ]:
numericitems_df = pd.read_csv('numericitems.csv', usecols=['admissionid', 'itemid', 'valueid'])

# See https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/concepts/lifesupport/mechanical_ventilation.ipynb
# itemid: valueids
list_items = {                                                                      ## Device + settings ##
              9534: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],                    # Type beademing Evita 1
              6685: [1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 20, 22]                  # Type Beademing Evita 4
              8189: [16],                                                           # Toedieningsweg O2
              12290: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],  # Ventilatie Mode (Set) - Servo-I and Servo-U ventilators
              12347: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],  # Ventilatie Mode (Set) (2) Servo-I and Servo-U ventilators
              12376: [1, 2],                                                        # Mode (Bipap Vision)
             }

# Create DataFrame corresponding to each itemid
patients_with_vent = []
for itemid, value_ids in tqdm(list_items.items()):
    condition = (numericitems_df['itemid'] == itemid) & (numericitems_df['valueid'].isin(value_ids))
    patients_with_vent.extend(numericitems_df[condition]['admissionid'].tolist())

print('Num patients on ventilator:', len(patients_with_vent))

In [ ]:
# Ventilator
vent = patient_df['admissionid'].isin(patients_with_vent)
print('Done!')

#### SIRS (on-admission)

In [ ]:
# todo

#### SOFA (on-admission)

In [ ]:
# todo

#### Save as `demographics_cohort.csv`

In [ ]:
# demographics_cohort.csv -> icustay_id, age, is_male, height, weight, vent, sofa, sirs
pd.DataFrame({'icustay_id': icustay_id,
              'age': age,
              'is_male': is_male,
              'height': height,
              'weight': weight,
              'vent': vent,
              'sofa': None,
              'sirs': None}).to_csv(OUTPUT_DIR + 'demographics_cohort.csv');
print('Done!')

---
## Vitals


In [46]:
# Item names
vital_itemids = {                                            ### For details: https://github.com/AmsterdamUMC/AmsterdamUMCdb/tree/master/amsterdamumcdb/sql/common
    'HeartRate': [6640],                                     # 'Hartfrequentie'
    'SysBP':     [6641, 6678, 8841],                         # 'ABP systolisch', 'Niet invasieve bloeddruk systolisch', 'ABP systolisch II', 
    'DiasBP':    [6643, 6680, 8842],                         # 'ABP diastolisch', 'Niet invasieve bloeddruk diastolisch', 'ABP diastolisch II'
    'MeanBP':    [6642, 6679, 8843],                         # 'ABP gemiddeld', 'Niet invasieve bloeddruk gemiddeld', 'ABP gemiddeld II'
    'Glucose':   [6833],                                     # 'Glucose Bloed'
    'SpO2':      [12311],                                    # 'O2-Saturatie (bloed)'
    'TempC':     [8658, 8659, 8662, 13058, 13059,            # 'Temp Bloed', 'Temperatuur Perifeer 2', 'Temperatuur Perifeer 1', 'Temp Rectaal', 'Temp Lies',
                 13060, 13061, 13062, 13063, 13952, 16110],  # 'Temp Axillair', 'Temp Oraal', 'Temp Oor', 'Temp Huid', 'Temp Blaas', 'Temp Oesophagus' 
    'RespRate':  [8874, 8873, 9654, 7726, 12266]             # 'Ademfrequentie Monitor', 'Ademfrequentie Evita', 'Ademfreq.'
}

In [ ]:
# Load numericitems.csv and convert to MIMIC-III namespace
numericitems_df = pd.read_csv('numericitems.csv', usecols=['admissionid', 'measuredat', 'itemid', 'value'])
numericitems_df.rename(columns={'admissionid': 'icustay_id', 
                                'measuredat': 'charttime',
                                'itemid': 'item_id',
                                'value': 'valuenum'}, inplace=True)

# Process vitals one-by-one
vital_dfs = []
for itemid, key in tqdm(vital_itemids.items()):
    condition = numericitem_df[numericitem_df['item_id'].isin(key)]
    vital_df = numericitems_df[condition][['icustay_id', 'charttime', 'valuenum']]
    vital_df['vital_id'] = itemid
    
    vital_dfs.append(vital_df)
    
vital_df = pd.concat(vital_dfs, axis=0)
print('Done!')

#### Save as `vitals_cohort.csv`

In [ ]:
# vitals_cohort.csv -> icustay_id, charttime, vital_id, valuenum
vitals_df.to_csv(OUTPUT_DIR + 'vitals_cohort.csv')
del vitals_df
print('Done!')

---
## Lab Results

In [28]:
dct = pd.read_csv('../data/amsterdam-umc-db/dictionary.csv')
dct[ (dct['item'].str.contains('PCO2'))]

,itemid,item,item_en,vocabulary_id,vocabulary_concept_code,vocabulary_concept_name,abbreviation,categoryid,category,category_en,...,unitid,unit,ucum_code,low_normal_value,high_normal_value,expected_min_value,expected_max_value,table,count,count_validated
167,6846,PCO2,partial pressure of carbon dioxide in blood,LOINC,11557-6,Carbon dioxide [Partial pressure] in Blood,PCO2,372,LAB ASTRUPS,NaN,...,173.0,mmHg,mm[Hg],36,44,NaN,NaN,numericitems,25348,25348.0
8985,21213,PCO2 (bloed) - kPa,partial pressure of carbon dioxide in blood,LOINC,11557-6,Carbon dioxide [Partial pressure] in Blood,PCO2 (bloed) - kPa,491,LAB-Astr-bloed,NaN,...,152.0,kPa,kPa,"4,7",6,NaN,NaN,numericitems,9464,9464.0


In [ ]:
lab_results = {
    'Calcium':     [9933],                           # 'Calcium', 'Calcium totaal (bloed)'
    'IonCalcium':  [],
    'ASAT':        [11990],                          # 'ASAT (bloed)'
    'PTT':         [17982],                          # 'APTT (bloed)'
    'Potassium':   [9927],                           # 'Kalium (bloed)'
    'PT':          [],
    'Platelet':    [6797, 9964, 10409, 14252, 7369], # 'Thrombocyten', "Thrombo's (bloed)", "Thrombo's citr. bloed (bloed)", 'Thrombo CD61 (bloed)'
    'AnionGap':    [9559],                           # 'Anion-Gap (bloed)'
    'PaO2':        [7433, 9996, 21214],              # 'PaO2', 'PaO2 (bloed)', 'PaO2 (bloed) - kPa'
    'ALAT':        [6800, 11978],                    # 'ALAT', 'ALAT (bloed)'
    'WBC':         [11678, 18553, 18554,             # 'Tot.WBC*10^8 (overig)', 'Tot. WBC*10^8 (HPC-A Allogeen)', 'Tot. WBC*10^8 (HPC-A Autoloog)'
                    18557, 6779, 9965],              # 'Tot. WBC*10^8 (Tcellen concentraat)', 'Leucocyten 10^9/l', "Leuco's (bloed) 10^9/l"
    'Bilirubin':   [9945, 6813],                     # 'Bilirubine (bloed)', 'Bili Totaal'
    'Sodium':      [12233, 9555, 9924, 10284],       # 'Natrium (overig)', 'Natrium Astrup', 'Natrium (bloed)', 'Na (onv.ISE) (bloed)'
    'Chloride':    [],
    'Magnesium':   [9952],                           # 'Magnesium (bloed)'
    'Lactate':     [10053],                          # 'Lactaat (bloed)'
    'PaCO2':       [6846, 9990, 21213],              # 'PCO2', 'PCO2 (bloed)', 'PCO2 (bloed) - kPa'
    'Glucose':     [6833, 9947],                     # 'Glucose Bloed', 'Glucose (bloed)'
    'Creatinine':  [9941, 14216],                    # 'Kreatinine (bloed)', 'KREAT enzym. (bloed)'
    'Bicarbinate': [],
    'BUN':         [],
    'pH':          [6848],                           # 'pH (bloed)'
    'Albumin':     [9975],                           # 'Albumine (imm.) (bloed)'
    'Bands':       [],
    'Hemoglobin':  [9960],                           # 'Hb (bloed)'
    'BaseExcess':  []
}

# Find lab results one-by-one
lab_dfs = []
for lab_id, keys in tqdm(lab_results.items()):
    condition = numericitem_df[numericitem_df['item_id'].isin(keys)]
    lab_df = numericitems_df[condition][['icustay_id', 'charttime', 'valuenum']]
    lab_df['lab_id'] = lab_id
    
    # Unit conversions
    lab_df[lab_df['item_id'] == 17982] *= 2           # APPT -> PTT: https://www.webmd.com/a-to-z-guides/partial-thromboplastin-time-test
    lab_df[lab_df['item_id'] == 21214] *= 7.50061683  # kPa -> mmHg; https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/common/pO2_FiO2_estimated.sql
    lab_df[lab_df['item_id'] == 21213] *= 7.50061683  # kPa -> mmHg; "
    
    lab_dfs.append(lab_df)
    
lab_df = pd.concat(lab_dfs, axis=0)
print('Done!')

#### Save as `labs_cohort.csv`

In [ ]:
# labs_cohort.csv -> icustay_id, charttime, lab_id, valuenum
lab_df = lab_df.to_csv(OUTPUT_DIR + 'labs_cohort.csv')
del lab_df
print('Done!')

In [ ]:
# dct = pd.read_csv('../data/amsterdam-umc-db/dictionary.csv')
# dct[ (dct['item'].str.contains('Dopamine'))]

---
## Urine Output

In [1]:
urine_output_ids = [
    8794,  # UrineCAD
    8796,  # UrineSupraPubis
    8798,  # UrineSpontaan
    8800,  # UrineIncontinentie
    8803,  # UrineUP
    10743, # Nefrodrain li Uit
    10745, # Nefrodrain re Uit
    19921, # UrineSplint Li
    19922  # UrineSplint Re
]

In [ ]:
# Limit measurements to Urine Output IDs
urine_df = numericitems_df[numericitems_df['itemid'].isin(urine_output_ids)]

#### Save as `urineoutput_cohort.csv`

In [ ]:
# urineoutput_cohort.csv -> icustay_id, charttime, valuenum
urine_df = urine_df.to_csv(OUTPUT_DIR + 'urineoutput_cohort.csv')
del urine_df
print('Done!')

---
## FiO2

**TODO:**

In [ ]:
# FiO2 on respiratory support (use if available)
fio2_resp_settings = [6699,  # FiO2 %: setting on Evita ventilator
                      12279, # O2 concentratie --measurement by Servo-i/Servo-U ventilator
                      12369, # SET %O2: used with BiPap Vision ventilator
                      16246] # Zephyros FiO2: Non-invasive ventilation

# Oxygen Flow settings without respiratory support
fio2_item_ids = [8845,  # O2 l/min
                 10387, # Zuurstof toediening (bloed)
                 18587] # Zuurstof toediening

Determine rows in `numericitems.csv` and `listitems` corresponding to FiO2:

In [ ]:
# Get listitems corresponding to O2
listitems_df = pd.read_csv('listitems.csv', usecols=['admissionid', 'measuredat'])
listitems_df = listitems_df[listitems_df['itemid'] == 8189] # Toedieningsweg (Oxygen device)

# Get numericitems corresponding to O2
fio2_non_vent_df = numericitems_df[numericitems_df['itemid'].isin(fio2_resp_settings + fio2_item_ids)]

# Left-join tables
fio2_df = fio2_non_vent_df.join(listitems_df, on=['admissionid', 'measuredat'])

**Case 1.** FiO2 according to respiratory settings on ventilator (see [sql version](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/common/pO2_FiO2_estimated.sql)):

In [ ]:
# FiO2 settings according to ventilator settings
fio2_vent_df = fio2_df[fio2_df['itemid'].isin(fio2_resp_settings)][['admissionid', 'measuredat', 'value']]
fio2_vent_df['value'].value(0.21)

**Case 2.** Estimated from `listitems` (see [sql version](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/common/pO2_FiO2_estimated.sql)):

In [ ]:
case1 = fio2_df[fio2_df['valueid'].isin([2, 7])] # nasaal + O2-bril
case1[]

#### Save as `fio2_cohort.csv`

---
## IV Fluids

In [ ]:
# Drug items contains all we need
drugitems_df = pd.read_csv('drugitems.csv', usecols=['admissionid', 'item', 'ordercategory', 'orderid', 'rate', 'rateunit', 'administered', 
                                                     'administeredunit', 'start', 'stop', 'fluidin', 'doserateperkg', 'doseunitid', 'doserateunitid'])

In [ ]:
# Which medications to consider IV fluids?
iv_fluid_categories = ['2. Spuitpompen', 
                       'Infuus - Colloid', 
                       'Infuus - Crystalloid', 
                       'Injecties Circulatie/Diuretica']

# Limit order categories to IV fluids
iv_fluid_df = drugitems_df[drugitems_df['ordercategoryid'].isin(iv_fluid_categories)]

#### Save as `inputevents_mv_cohort.csv`

In [ ]:
# labs_cohort.csv -> icustay_id, charttime, lab_id, valuenum
iv_fluid_df = iv_fluid_df.rename(columns={'admissionid': 'icustay_id', 
                                          'start': 'starttime',
                                          'end': 'endtime',
                                          'rateunit': 'rateuom',
                                          'administered': 'amount',
                                          'administeredunit': 'amountuom'}).to_csv(OUTPUT_DIR + 'inputevents_mv_cohort.csv')
del iv_fluid_df
print('Done!')

---
## Vasopressors

In [ ]:
vasopressors = ['Dopamine (Inotropin)', 
                'Dobutamine (Dobutrex)', 
                'Adrenaline (Epinefrine)', 
                'Noradrenaline (Norepinefrine)']

In [ ]:
# Limit drugitems to vasopressors
vaso_df = drugitems_df[(drugitems_df['orderid'] == 65) & (drugitems_df['itemid'].isin(vasopressors))] # orderid = 65 -> continuous i.v. perfusor

# Add patient weight to vasopressor DataFrame
vaso_df = vaso_df.join(icustay_id.to_frame().join(weight), on='admissionid')

# Convert mcg/min to mcg/kg/min
def convert_vaso_units(row):
    # doserateperkg:  Whether dose is already per kg
    # doseunitid:     10 -> mg;  11 -> µg
    # doserateunitid:  4 -> min;  5 -> uur
    
    # µg/min -> µg/kg/min
    if not row['doserateperkg'] and row['doseunitid'] == 11 and row['doserateunitid'] == 4:
        return row['dose'] / row['weight']
    
    # mg/min -> µg/kg/min
    if not row['doserateperkg'] and row['doseunitid'] == 10 and row['doserateunitid'] == 4:
        return 1000 * row['dose'] / row['weight']
    
    # mg/uur -> µg/kg/min
    if not row['doserateperkg'] and row['doseunitid'] == 10 and row['doserateunitid'] == 5:
        return 1000 * row['dose'] / row['weight'] / 60
    
    # µg/kg/min! 
    if row['doserateperkg'] and row['doseunitid'] == 11 and row['doserateunitid'] == 4:
        return row['dose']
    
    # µg/kg/uur -> µg/kg/min
    if row['doserateperkg'] and row['doseunitid'] == 11 and row['doserateunitid'] == 5:
        return row['dose'] / 60
    return row['dose']
    
vaso_df['rate'] = vaso_df.transform(convert_vaso_units)
print('Done!')

#### Save as `vasopressors_mv_cohort.csv`

In [ ]:
# labs_cohort.csv -> icustay_id, charttime, lab_id, valuenum
vaso_df = vaso_df.rename(columns={'admissionid': 'icustay_id', 
                                  'start': 'starttime',
                                  'end': 'endtime',
                                  'rateunit': 'rateuom',
                                  'administered': 'amount',
                                  'administeredunit': 'amountuom'}).to_csv(OUTPUT_DIR + 'vassopressors_mv_cohort.csv')
del vaso_df
print('Done!')